## Treinamento do modelo de `Who X also X`
Treinamento do modelo  de estudo paseado na interação de visualização de produtos num e-commerce, assim  criando um modelo de recomendação `quem-viu-tambem-viu`

### Importação da bibliotecas

In [ ]:
import pandas as pd
import numpy as np

### Lendo DataSets de produtos e interações

In [ ]:
df_products = pd.read_csv('products.csv')

df_interactions = pd.read_csv('interactions.csv')
df_interactions.head()

### Gerando Matriz de Usuarios x Produtos vistos

In [ ]:
df_interactions = df_interactions[['customer_id', 'product_id', 'action_id']]
users_items_matrix_df = df_interactions.pivot_table(
    index='customer_id',
    columns='product_id',
    values='action_id',
).fillna(0)
users_items_matrix_df.head()

### Modelo Keras com encoder e decoder

Implementão do modelo conténdo um layer no encoder e um no decoder de tamanho 512, o espaço-latente no centro do modelo tem 256 dimensões.

In [ ]:
from keras.optimizers import Adam
from keras.layers import Input, Dense, Dropout
from keras.models import Model

def autoEncoder(X):
  '''
  Autoencoder for Collaborative Filter Model
  '''

  # Input
  input_layer = Input(shape=(X.shape[1],), name='UserScore')

  # Encoder
  # -----------------------------
  enc = Dense(512, activation='selu', name='EncLayer1')(input_layer)

  # Latent Space
  # -----------------------------
  lat_space = Dense(256, activation='selu', name='LatentSpace')(enc)
  lat_space = Dropout(0.8, name='Dropout')(lat_space) # Dropout

  # Decoder
  # -----------------------------
  dec = Dense(512, activation='selu', name='DecLayer1')(lat_space)

  # Output
  output_layer = Dense(X.shape[1], activation='linear', name='UserScorePred')(dec)

  # this model maps an input to its reconstruction
  model = Model(input_layer, output_layer)    

  return model

### Instanciando o modelo

In [ ]:
# input
X = users_items_matrix_df.values
y = users_items_matrix_df.values

# Build model
model = autoEncoder(X)
model.compile(optimizer = Adam(lr=0.0001), loss='mse')

model.summary()

### Treinando o modelo

In [ ]:
# Train
hist = model.fit(x=X, y=y,
                  epochs=50,
                  batch_size=64,
                  shuffle=True,
                  validation_split=0.1)

### Recomendação de Conteúdo para o Usuário

In [ ]:
# Predict new Matrix Interactions, set score zero on visualized games
new_matrix = model.predict(X) * (X == 0)

# converting the reconstructed matrix back to a Pandas dataframe
new_users_items_matrix_df  = pd.DataFrame(new_matrix, 
                                          columns = users_items_matrix_df.columns, 
                                          index   = users_items_matrix_df.index)
new_users_items_matrix_df.head()

In [ ]:
def recommender_for_user(user_id, interact_matrix, df_content, topn = 10):
    '''
    Recommender Games for UserWarning
    '''
    pred_scores = interact_matrix.loc[user_id].values

    df_scores   = pd.DataFrame({'product_id': list(users_items_matrix_df.columns), 
                               'score': pred_scores})

    df_rec      = df_scores.set_index('product_id')\
                    .join(df_content.set_index('product_id'))\
                    .sort_values('score', ascending=False)\
                    .head(topn)[['score']]
    
    return df_rec[df_rec.score > 0]

In [ ]:
# Recommended User Games
recommender_for_user(user_id         = 'ID-30', 
                     interact_matrix = new_users_items_matrix_df, 
                     df_content      = df_products)